# Analyse outillée d'une correspondance en XML-TEI

Ce notebook vise à présenter quelques méthodes d'analyse computationnelle, d'encodage automatique et de visualisation de données. D'un point de vue technique, cet atelier introduit au XML-TEI, et surtout à son analyse outillée avec Python:
- lecture, extraction d'informations, construction et écriture de fichiers XML (librairie `lxml`)
- enrichissement automatique par la récupération d'informations géographiques via des API (librairie `requests`)
- extraction d'informations: classification automatisée d'entités nommées (TAL, avec `spacy`)
- visualisation en réseau du corpus (librairie `pyvis`)
- cartographie interactive (librairie `folium`).

Le corpus utilisé pour cet atelier est un ensemble de correspondances autour de l'achat par Matsukata d'un ensembles d'œuvres européennes, notamment par l'intermédiaire de Léonce Bénédite, durant la première moitié du XIXe siècle. Les originaux sont conservés dans les archives de l'Institut national d'histoire de l'art et du Musée Rodin. La version numérique du corpus a été produite par Léa Saint-Raymond (production d'une version en texte brut avec indexation des métadonnées). À partir de là, un encodage automatique a été réalisé en XML-TEI, à l'aide de Python (voir le script [txt2xml.py](https://github.com/paulhectork/cours_ens2023_xmltei/blob/main/src/txt2xml.py)).

À partir de ce corpus, nous allons:
- extraire des informations géographiques (lieu d'expédition/réception) et les géocoder à l'aide de données d'Openstreetmap. On se servira de ces données pour produire un `settingDesc` dans l'`encodingDesc` du `teiHeader` de chaque fichier XML.
- identifier certaines entités (personnes et organisations expéditrices/destinataires de lettres du corpus), et classer celles-ci à l'aide de de Spacy (reconnaissance d'entités nommées, apprentissage machine). À partir de là, on construira un `particDesc` qui documente tou.te.s les expéditeur.ice.s et destinataires de lettres du corpus.
- produire une visualisation en réseau de ces expéditeur.ice.s et destinataires, en faisant de la fouille de texte du corpus de fichiers XML
- produire une cartographie interactive des villes d'expédition/réception de lettres du corpus, là encore en faisant de la fouille de texte.

Tout un programme donc!

---

## La structure des fichiers XML

Les fichiers XML produits à la fin de l'exercice ressembleront à l'exemple ci-dessous. Plusieurs éléments ne sont pas présents dans les fichiers au début de l'exercice, et surtout les `settingDesc` et `particDesc`.

```xml
<TEI xmlns="http://www.tei-c.org/ns/1.0" 
     xml:id="KojiroMatsukata_L&#233;onceB&#233;n&#233;dite_19200619_4832865354280595070">
  <!-- @xmlns: l'escpace de nom de la TEI -->
  <!-- @xml:id: l'identifiant de notre fichier, qui correspond avec le nom de celui-ci -->
  <teiHeader>
    <!-- teiHeader: l'en-t&#234;te de notre document, contenant les m&#233;tadonn&#233;es -->
    <fileDesc>
      <!-- fileDesc: description du fichier encod&#233; et de sa source -->
      <titleStmt>
        <!-- titleStmt: informations sur le titre du document -->
        <title>Lettre typographi&#233;e sur papier &#224; en-t&#234;te Kawasaki, Kobe, de Matsukata &#224; Leone </title>
        <author>Kojiro Matsukata</author>
        <respStmt>
          <!-- respStmt: qui sont les responsables de la production de l'encodage numérique -->
          <resp>Production et pr&#233;paration du texte brut</resp>
          <persName>L&#233;a Saint-Raymond</persName>
        </respStmt>
        <respStmt>
          <resp>Transformation automatique du texte brut vers le XML-TEI</resp>
          <persName>Les participant.e.s &#224; l'atelier "Mod&#233;liser et exploiter des corpus textuels" (ENS-PSL, campus d'Ulm)</persName>
        </respStmt>
      </titleStmt>
      <publicationStmt>
        <!-- piblicationStmt: informations sur le document XML encodé -->
        <publisher>ENS-PSL</publisher>
        <pubPlace>Paris (France)</pubPlace>
        <date>2023-03-26 16:02:14.447940</date>
      </publicationStmt>
      <sourceDesc>
        <!-- sourceDesc: informations sur la source 
             (l'original papier, la lettre que l'on encode) -->
        <bibl type="lettre">
          <author>Kojiro Matsukata</author>
          <title>Lettre typographi&#233;e sur papier &#224; en-t&#234;te 
              Kawasaki, Kobe, de Matsukata &#224; Leone </title>
          <date>1920-06-19</date>
          <msIdentifier>
            <institution>INHA</institution>
            <idno>INHA 56</idno>
          </msIdentifier>
        </bibl>
      </sourceDesc>
    </fileDesc>
    <encodingDesc>
      <!-- informations sur l'encodage :
           - tei:editorialDecl, décrivant la méthode de production du document XML 
           - tei:projectDesc, contexte de production du XML 
      -->
      <editorialDecl>
        <p>Production de l'encodage XML r&#233;alis&#233;e automatiquement avec la librairie LXML de Python &#224; partir d'une version en texte brut de la correspondance de Matsukata</p>
      </editorialDecl>
      <projectDesc>
        <p>L'atelier "Mod&#233;liser et exploiter des corpus textuels" a donn&#233; lieu &#224; cet encodage.</p>
      </projectDesc>
    </encodingDesc>
    <profileDesc>
      <!-- informations non bibliographiques sur le document encod&#233; 
           cet élément sera complété au fil du notebook par:
           - un tei:particDesc, qui décrit tout.es les personnes ayant écrit
             ou reçu des lettres dans le corpus Matsutaka
           - un tei:settingDesc, qui décrit tous les lieux d'expédition/destination
             des lettres du corpus Matsutaka
           ces éléments seront rajoutés dans le notebook du cours.
      -->
      <correspDesc>
        <!-- correspAction: description de la correspondance -->
        <correspAction type="sent">
          <persName ref="#kojiromatsukata">Kojiro Matsukata</persName>
          <placeName ref="#kobe">Kobe</placeName>
          <date when="1920-06-19">1920-06-19</date>
        </correspAction>
        <correspAction type="received">
          <persName ref="#leoncebenedite">L&#233;once B&#233;n&#233;dite</persName>
          <placeName ref="#paris">Paris</placeName>
        </correspAction>
      </correspDesc>
      <settingDesc>
        <!-- le listPlace sera créé pendant le notebook: il décrit 
             les lieux  avec la structure suivante: -->
        <listPlace>
          <place xml:id="paris">
            <placeName>Paris</placeName>
            <location>
              <geo>2.3483915 48.8534951</geo>
            </location>
          </place>
          <!-- ... -->
        </listPlace>
        -->
      </settingDesc>
      <particDesc>
        <!-- pareil, le particDesc sera créé au fil du notebook. il aura une structure
             équivalente: -->
        <listPerson>
          <person type="PERSON" xml:id="leoncebenedite">
            <persName>L&#233;once B&#233;n&#233;dite</persName>
            <persName>L&#233;once B&#233;n&#233;dite ?</persName>
          </person>
          <!-- ... -->
        </listPerson>
        <listOrg>
          <org type="ORG" xml:id="compagniealgerienne">
            <orgName>Compagnie alg&#233;rienne</orgName>
          </org>
          <!-- ... -->
        </listOrg>
      </particDesc>
    </profileDesc>
  </teiHeader>
  <text>
    <!-- le corps du texte. il peut comporter un tei:front et un tei:back
         et doit comporter un tei:body, comprenant le corps du texte -->
    <body>
      <opener>
        <!-- le corps du texte -->
        <salute>Dear Sir, </salute>
      </opener>
      <p><!-- la lettre en tant que telle --></p>
      <closer>
        <!-- la fermeture -->
        <salute>Yours faithfully, </salute>
        <signed>Kojiro Matsukata</signed>
      </closer>
    </body>
  </text>
</TEI>
```

---

## Les bases

Pour bien faire notre travail, on commence par **importer les librairies**. Une librairie est un ensemble de fonctions avec une finalité spécifique: visualisation, fouille de texte... Certaines viennent par défaut, d'autres sont conçues par des tiers pour augmenter les fonctionnalités de Python et doivent être installées (ce qu'on a fait avec `pip install`).

In [ ]:
# installer les dépendances. ça va prendre du temps
!pip install asttokens==2.2.1
!pip install backcall==0.2.0
!pip install blis==0.7.9
!pip install branca==0.6.0
!pip install catalogue==2.0.8
!pip install certifi==2022.12.7
!pip install charset-normalizer==3.1.0
!pip install click==8.1.3
!pip install cmake==3.26.1
!pip install confection==0.0.4
!pip install cymem==2.0.7
!pip install decorator==5.1.1
!pip install en-core-web-trf@https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.5.0/en_core_web_trf-3.5.0-py3-none-any.whl
!pip install executing==1.2.0
!pip install filelock==3.10.7
!pip install folium==0.14.0
!pip install fr-dep-news-trf@https://github.com/explosion/spacy-models/releases/download/fr_dep_news_trf-3.5.0/fr_dep_news_trf-3.5.0-py3-none-any.whl
!pip install huggingface-hub==0.13.3
!pip install idna==3.4
!pip install ipython==8.11.0
!pip install jedi==0.18.2
!pip install Jinja2==3.1.2
!pip install jsonpickle==3.0.1
!pip install langcodes==3.3.0
!pip install langdetect==1.0.9
!pip install lit==16.0.0
!pip install lxml==4.9.2
!pip install MarkupSafe==2.1.2
!pip install matplotlib-inline==0.1.6
!pip install mpmath==1.3.0
!pip install murmurhash==1.0.9
!pip install networkx==3.0
!pip install numpy==1.24.2
!pip install nvidia-cublas-cu11==11.10.3.66
!pip install nvidia-cuda-cupti-cu11==11.7.101
!pip install nvidia-cuda-nvrtc-cu11==11.7.99
!pip install nvidia-cuda-runtime-cu11==11.7.99
!pip install nvidia-cudnn-cu11==8.5.0.96
!pip install nvidia-cufft-cu11==10.9.0.58
!pip install nvidia-curand-cu11==10.2.10.91
!pip install nvidia-cusolver-cu11==11.4.0.1
!pip install nvidia-cusparse-cu11==11.7.4.91
!pip install nvidia-nccl-cu11==2.14.3
!pip install nvidia-nvtx-cu11==11.7.91
!pip install packaging==23.0
!pip install parso==0.8.3
!pip install pathy==0.10.1
!pip install pexpect==4.8.0
!pip install pickleshare==0.7.5
!pip install preshed==3.0.8
!pip install prompt-toolkit==3.0.38
!pip install protobuf==3.20.3
!pip install ptyprocess==0.7.0
!pip install pure-eval==0.2.2
!pip install pydantic==1.10.7
!pip install Pygments==2.14.0
!pip install pyvis==0.3.2
!pip install PyYAML==6.0
!pip install regex==2023.3.23
!pip install requests==2.28.2
!pip install sentencepiece==0.1.97
!pip install six==1.16.0
!pip install smart-open==6.3.0
!pip install spacy==3.5.1
!pip install spacy-alignments==0.9.0
!pip install spacy-legacy==3.0.12
!pip install spacy-loggers==1.0.4
!pip install spacy-transformers==1.2.2
!pip install srsly==2.4.6
!pip install stack-data==0.6.2
!pip install sympy==1.11.1
!pip install thinc==8.1.9
!pip install tokenizers==0.13.2
!pip install torch==2.0.0
!pip install tqdm==4.65.0
!pip install traitlets==5.9.0
!pip install transformers==4.26.1
!pip install triton==2.0.0
!pip install typer==0.7.0
!pip install typing_extensions==4.5.0
!pip install Unidecode==1.3.6
!pip install urllib3==1.26.15
!pip install wasabi==1.1.1
!pip install wcwidth==0.2.6

In [ ]:
from pyvis.network import Network  # réseaux
from langdetect import detect      # détection de la langue
from statistics import mode        # statistique
from zipfile import ZipFile        # zipper / dézipper des fichiers
from lxml import etree             # traiter du xml
import unidecode                   # opérations sur du texte
import requests                    # requêtes HTTP sur le Web
import folium                      # cartographie
import shutil                      # déplacements de fichiers
import spacy                       # traitement automatisé des langues
import json                        # traitement des fichiers json
import time                        # le temps
import math                        # maths (obviously)
import sys                         # opérations sur le système d'exploitation
import re                          # expressions régulières
import os                          # opérations sur les fichiers et le système d'exploitation

Ensuite, on **définit nos variables globales**. Celles-ci ont différents usages (principalement des chemins de fichiers), ne seront pas modifiées, et seront utilisées à différents endroits de notre code. 

In [ ]:
# CURDIR = os.path.abspath(os.path.dirname(""))                                  # dossier actuel
# TXT = os.path.abspath(os.path.join(CURDIR, os.pardir, "txt"))                  # dossier `txt/`
WEB = os.path.abspath(os.path.join("/", "content", "web"))                       # dossier `web/`
XML = os.path.abspath(os.path.join("/", "content", "xml"))                       # dossier `xml/`
UNZIP = os.path.abspath(os.path.join(XML, "unzip"))                              # dossier `xml/unzip`
NS_TEI = {"tei": "http://www.tei-c.org/ns/1.0"}                                  # tei namespace
NS_XML = {"id": "http://www.w3.org/XML/1998/namespace"}                          # general xml namespace
TEI_RNG = "https://tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng"  # odd in .rng to validate tei files
PARSER = etree.XMLParser(remove_blank_text=True)                                 # parser xml custom
COLORS = { "green": "#8fc7b1",                                                   # codes couleurs html 
           "gold": "#da9902", 
           "plum": "#710551", 
           "darkgreen": "#00553e" }

On **crée nos dossiers d'entrée et de sortie** pour ce faire, on utilise la librairie OS, à la syntaxe peu élégante, mais très pratique:

In [ ]:
# les chemins de fichiers sont exprimés depuis la base du dépôt, soit depuis le dossier parent
# si le dossier `xml/unzip` n'existe pas, on le crée
if not os.path.isdir(UNZIP):
    os.makedirs(UNZIP)
# si le dossier `web/` et `web/json` n'existent pas, on les crée
if not os.path.isdir(os.path.join(WEB, "json")):
    os.makedirs(os.path.join(WEB, "json"))

On **dézippe nos fichiers XML**. Ceux-ci sont embarqués dans une archive Zip pour faciliter l'atelier (et surtout pour l'utilisation de Google Colab). 

In [ ]:
# on dézippe nos fichiers
with ZipFile( os.path.join(XML, "corpus_matsutaka.zip"), mode="r" ) as zip:
    zip.extractall(path=UNZIP)

# on crée une dernière variable globale: une liste de tous les chemins absolus
# vers les fichiers XML encodés. La liste permettra d'accéder aux fichiers.
CORPUS = [ os.path.join(UNZIP, f) for f in os.listdir(UNZIP) ]

On crée enfin une fonction qui permette de **créer un @xml:id** à partir d'une chaîne de caractères. Elle sera utilisée à plusieurs points du corpus.

Pour rappel, la syntaxe pour définir une fonction est la suivante:

```python
def nom_de_fonction(x):
    """
    documentation
    """
    # opérations python
    x = x*x  
    return x
```

Deux petites choses à ce sujet:
- Les *paramètres* (ici `x`), définis après le nom de la fonction entre parenthèses, sont un ensemble de valeurs données en entrée à la fonction. Quand on active la fonction, on peut donc la faire s'exécuter avec des valeurs spécifiques.
- Le `return` est la sortie de la fonction et permet de "retourner" une valeur: à la fin de l'exécution d'une fonction, Python supprime toutes les variables créées pendant l'exécution. Comme en maths, les valeurs retournées sont le "résultat" de la fonction, et seront donc utilisables en dehors de la fonction.

Pour appeler une fonction, on fait:

```python
nom_de_fonction(2)  # les `()` indiquent que l'on appelle la fonction. les valeurs entre parenthèses sont des paramètres: on calculera ici le carré de 2
```

In [ ]:
# il y a une tripotées de syntaxes peu claires ici mais ce n'est pas un problème central
def xmlid(text):
    """
    fonction pour créer un @xml:id à partir de la chaîne de caractère `text`.
    permet de normaliser la création d'identifiants uniques. on ne garde que
    les caractères alphanumériques sans majuscules de `text` + on enlève les
    accents des lettres avec `unidecode`.
    
    :param text: le texte à partir duquel produire un identifiant
    :returns: l'identifiant
    """
    return unidecode.unidecode("".join( c for c in text.lower() if c.isalnum()))

---

## Enrichissements automatiques 

### Les données géographiques

Ici, on va faire 3 opérations:
- extraire de tout le corpus l'ensemble des **lieux d'expédition/destination des lettres**. Ensuite, attribuer à chaque lieu un @xml:id avec la fonction définie ci-dessus
- à partir du nom des lieux, lancer des **requêtes sur l'API [nominatim](https://nominatim.org/release-docs/develop/api/Search/)** pour géocoder nos lieux d'expédition/réception.
- **créer un `settingDesc`**, un élément TEI dans le `teiHeader` qui permette de décrire les lieux auxquels sont liés un corpus et de stocker leur géolocalisation.

#### 1. Définition des variables de base

À la fin de cette étape, on aura donc un corpus géolocalisé, et dont le géocodage sera documenté par les documents XML eux-mêmes! On commence par définir nos variavbles de base:

In [ ]:
place_list = []  # liste de lieux
endpoint = "https://nominatim.openstreetmap.org/search?"  # url de l'api nominatim    

C'est maintenant l'heure de notre première rencontre de LXML, la librairie pour parser et manipuler des fichiers XML. Ce n'est pas la librairie la plus simple à prendre en main (de loin), mais elle est très pratique pour de gros corpus.
- [*parser*](https://en.wikipedia.org/wiki/Parsing), en informatique, c'est faire lire un fichier à un ordinateur de manière à ce qu'il comprenne et valide sa structure. Parser du XML, c'est donc rendre sa structure *machine readable*. L'ordinateur pourra faire la distinction entre les éléments, comprendre leur imbrication...

On crée deux variables qui stockent chacune un objet `lxml` décrivant un élément XML (balise+texte+imbrication+attributs...).
- la fonction `etree.Element()` crée un élément, la fonction `etree.SubElement()` crée un sous-élément d'un élément prééxistant
- l'argument `nsmap` permet de définir un espace de nom, une des petites complexités du XML: une sémantique spécifique (comme la TEI) doit être définie par un nom et rattachée à une URI (équivalent de l'URL). Par exemple, l'espace de nom TEI est associé à "http://www.tei-c.org/ns/1.0". Cela permet de localiser les vocabulaires.

In [ ]:
settingDesc = etree.Element(
    "settingDesc"   # nom de l'élément
    , nsmap=NS_TEI  # l'espace de noms auquel il appartient
)
listPlace = etree.SubElement(
    settingDesc      # l'élément parent, soit le `settingDesc` créé juste au dessus
    , "listPlace"    # le nom de l'élément
    , nsmap=NS_TEI   # son espace de nom
)

#### 2. Création d'une liste de lieux

On fouille notre corpus une première fois pour **construire une liste dédoublonnée de lieux d'expédition/réception de lettres**:
- on lit tous les fichiers vers lesquels pointe notre liste `CORPUS`
- on les parse avec `etree.parse(nom_de_fichier)`. Un fichier XML parsé est dit un *arbre*, vu sa structure arborescente
- on extrait les noms de lieux avec `etree.xpath(expression_xpath)` (on verra ça plus bas)
- on enlève certains caractères
- on construit enfin `place_list`: à chaque itération, si une des `place` d'expédition/destination n'est pas présente dans la liste, on l'y ajoute.

In [ ]:
for fpath in CORPUS:  # on accède à chaque chemin vers un fichier xml
    tree = etree.parse(fpath, parser=PARSER)  # on parse chaque fichier
    for place in tree.xpath(".//tei:correspAction/tei:placeName", namespaces=NS_TEI):  # on cible tous les lieux
        place = place.text.replace("?", "").strip()  # simplifier la chaîne de caractères
        if (
            place not in place_list 
            and not re.search("^(inconnu|aucun)$", place)
        ):
            # ajouter le lieu s'il n'est pas déjà dans la liste
            # en supprimant les notations équivalentes à "NA" (lieu inconnu)
            place_list.append(place)
    
print(place_list)  # on montre notre liste de lieux

Il s'est passé quoi?? Plus précisément, c'est quoi cette ligne??

```python
tree.xpath(".//tei:correspAction/tei:placeName", namespaces=NS_TEI)
```

`tree.xpath` permet **d'évaluer une expression `xPath` sur un arbre XML**. `xPath` est un langage assez compact et assez puissant pour naviguer à l'intérieur d'un document XML. Ce langage permet de traverser un document complexe, en sautant les éléments intermédiaires pour cibler ceux qui nous intéressent seulement. Parmi les principes:
- une `xPath` est structurée de façon analogue à un chemin de fichier ou à une URL: `elementA/elementB/elementC`...
- une xpath permet de cibler des *nœuds* à l'intérieur du document. Ces nœuds peuvent être:
    - des éléments XML, écrits sans préfixe `nom-de-l'élément` 
    - un attribut, préfixé une `@`: `@ref`
    - `*` représente "n'importe quel nœud"
    - `.` représente le nœud actuel
    - une fonction qui permet d'évaluer un résultat
- dans une xpath, des nœuds peuvent être ciblés en fonction de:
    - leur positionnement relatif: 
        - `nœudA/nœudB` permet de cibler un nœud B qui est l'enfant direct d'un nœud A.
        - `nœudA//nœudB` permet de cibler un nœud B qui est descendant du nœud A: il est inclus dans celui-ci, mais il peut y avoir plusieurs intermédiaires entre A et B.
        - il est aussi possible de traverser l'arbre dans des directions différentes: de l'enfant au parent...
    - des propriétés, écrites entre `[]`. 
        - Par exemple, `a[@xml:id="Artemisia"]` permet de cibler un élément `a` qui a un attribut `@xml:id` dont la valeur est `Artemisia`. 
        - on peut mettre bout à bout les propriétés: `*[@xml:id="Artemisia"][type="painter"]` permet de cibler n'importe quel élément dont l'`@xml:id` est "Artemisia" et le `@type` est "painter". 
- on peut bien sûr combiner propriétés, sélection d'éléments et d'attributs, ajouter des fonctions et même mélanger les directions dans lesquelles l'arbre est traversé pour arriver à des très structures complexes:
    ```
    body//rdg[not(ancestor::rdgGrp)][not(ancestor::app//app)]
    ```
        
En bref, `.//tei:correspAction/tei:placeName` permet de cibler tous les `placeName` dans un `correspAction`. Ces éléments doivent être descendants de l'élément actuel.

[Pour aller plus loin sur les xpath : )](https://github.com/paulhectork/tnah2021_cours/blob/main/export/cours_markdown/xquery-xpath_fiche.md), descendre jusqu'à arriver dans la partie `XPATH`.

#### 3. Géocodage des lieux

On a créé une liste de lieux où les lettres sont écrites ou envoyées. On peut maintenant **géocoder les lieux**, c'est-à-dire, à partir d'une addresse "lisible par les humain.e.s", obtenir des coordonnées. Pour cela, on utilise l'API Nominatim, qui permet d'accéder aux données d'Openstreetmap.

Mais d'abord: **une API, c'est quoi?** La manière la plus brève de définir une API Web, c'est de dire que c'est **un site internet pour machines**. Une API permet à une machine d'interagir avec un serveur à distance, de lui envoyer et de récupérer des données brutes, de façon automatique. 

Dans notre cas, l'API sera utilisée pour **faire une recherche en plein texte à notre place**, pour obtenir des coordonnées géographiques à partir d'un nom de ville. Plutôt que de faire la recherche à la main, on fait un script qui **effectue la recherche à notre place, stocke les résultats et met à jour nos documents XML.**

Pour que notre script communique avec uns serveur à distance, les APIs utilisent les mêmes standards que n'importe quels sites Web, et surtout le **HTTP(S)**. Ce standard permet:
- à un **client** (notre script) de poser une question sous la forme d'une URL à un serveur.
- à un **serveur** (Openstreetmap) de répondre en nous renvoyant des données correspondant à notre requête.

![client server architecture](https://miro.medium.com/v2/resize:fit:1400/0*9iZ6PlYHEOwi0-X-)

Pour faire nos requêtes, on utilise **la librairie `Request`.** Pour chaque lieu, 
- on construit un élément HTML `place` documentant le lieu
- on construit une URL pour faire une requête sur cette ville
- on lance la requête, récupère et sauvegarde le GeoJSON produit, et on ajoute les géocoordonnées au `place`.

[Pour plus d'infos sur les APIs : )](https://github.com/paulhectork/cours_ens2023_fouille_de_texte/blob/main/2_bonus_creation_corpus.ipynb)

In [ ]:
for placename in place_list:
    # créer un identifiant unique @xml:id
    idx = xmlid(placename)
        
    # créer l'élément tei `place`, contenu par `listPlace`
    # et qui contient toutes les informations sur le lieu
    place = etree.SubElement(
        listPlace
        , "place"
        , nsmap=NS_TEI
    )
    # définir son @xml:id
    place.set("{http://www.w3.org/XML/1998/namespace}id", idx)
    # y ajouter un sous élément `placeName` contenant le nom du lieu.
    etree.SubElement(
        place
        , "placeName"
        , nsmap=NS_TEI
    ).text = placename
    
    # jusque là tout va bien. on va maintenant commencer à faire des requêtes:
    if placename == "NA" or placename == "na":
        # continue stoppe ici l'itération actuelle 
        # => la requête API n'est pas lancée et on passe à l'item suivant
        continue
    
    time.sleep(1)  # il faut attendre 1s entre 2 requêtes
    
    # requests.get() lance une requête HTTP Get. on lui donne 2 arguments:
    # - `endpoint`: l'URL de pase de l'API
    # - `params`: un dictionnaire de paramètres. request construit une URL
    #    en combinant l'endpoint et les params.
    r = requests.get(endpoint, params={ 
        "city": placename,    # le nom de la ville recherchée
        "format": "geojson",  # format de la réponse: json
        "limit": 1            # nombre de résultats à afficher
    })
    
    # si on ne trouve pas de nom de ville, alors on fait une recherche 
    # libre en utilisant le param `q` à la place de `city`
    if len(r.json()) == 0:
        time.sleep(1)
        r = requests.get(endpoint, params={ 
            "q": placename,       # `q`: query, paramètre de recherche libre
            "format": "geojson",  # format de la réponse: geojson
            "limit": 1            # nombre de résultats à afficher
        })
        
    # les requêtes ont été lancées. 
    # si la requête s'est bien passée et qu'on a des résultats, alors
    # - on parse le geojson retourné pour extraire les informations
    # - on complète notre élément xml `place` avec un `location` qui
    #   contient un `geo`, celui-ci contenant les géocoordonnées
    # - on enregristre notre geojson dans le dossier `WEB/json/`
    print(r.url)    # on affiche l'url; une API fonctionne bien comme un site normal!
    res = r.json()  
    if r.status_code == 200 and len(res["features"]) > 0:  # si il n'y a pas d'erreur
            
        # on extrait les coordonnées et les ajoute à `place`
        if res["features"][0]["geometry"]["type"] == "Point":
            # on convertit la liste de coordonnées en string
            coordinates = "".join( 
                f"{c} " for c in res["features"][0]["geometry"]["coordinates"] 
            ).strip()
            # on crée notre élément xml
            location = etree.SubElement(
                place
                , "location"
                , nsmap=NS_TEI
            )
            etree.SubElement(
                location
                , "geo"
                , nsmap=NS_TEI
            ).text = coordinates
                
            # enfin, on enregistre notre json. la réponse est en geojson => on crée
            # un fichier geojson et on l'enregistre, pour pouvoir y accéder +tard
            with open( os.path.join(WEB, "json", f"{idx}.geojson"), mode="w" ) as fh:
                json.dump(res, fh, indent=4)
                
        else:
            # il n'y a pas de coordonnées, c'est étrange => on print
            print(f"pas de coordonnées pour '{placename}'", "\n", res, "\n\n")

**Dans le bloc de code précédent**, on a géocodé toutes les villes du corpus, construit notre `listPlace` et on a produit un paquet de fichiers `geojson` qui seront utiles pour notre visualisation. En faisant `print(r.url)`, on voit bien qu'une API fonctionne comme un site normal: on pose une question à un serveur, le serveur renvoie une réponse. La seule chose qui change, c'est le format.

#### 4. Mise à jour des fichiers XML

Toutes nos données sont prêtes. Il ne reste donc qu'à mettre à jour notre corpus de fichiers XML:
- on ajoute au `profileDesc` le `settingDesc` créé plus haut. 
- on met à jour le `correspAction` (qui décrit la correspondance): on ajoute un attribut `@ref` aux `correspAction/placeName` pour faire un renvoi entre cette information (envoi de la lettre spécifique) et le `listPlace`, qui centralise toutes nos informations spatiales.

Tout ça va être l'occasion de voir des xpath un peu plus complexes c:

In [ ]:
# on a pris l'habitude: on lit tous nos fichiers XML et on les parse
for fpath in CORPUS:
    tree = etree.parse(fpath, parser=PARSER)
    
    # ajout du `settingDesc`
    tree.xpath(".//tei:profileDesc", namespaces=NS_TEI)[0].append(settingDesc)
    
    # ajout des `@ref` aux `placeName`.
    # pour ce faire, on reconstruit l'`@xml:id` à partir de son orthographe => on peut
    # comparer entre l'identifiant créé ici (`placetext`) et l'`@xml:id` du `settingDesc`   
    for placename in tree.xpath(".//*[not(tei:settingDesc)]//tei:placeName", namespaces=NS_TEI):
        placetext = xmlid(placename.text)  # on reconstruit l'@xml:id
        
        # inconnu, aucun => on donne la valeur `#na` à `@ref`
        if re.search("^(inconnu|aucun)$", placetext):
            placename.set("ref", "#na")

        # pour tous les autres cas, on définit le `@ref`.
        # celui-ci est toujours préfixé d'un `#` pour montrer
        # qu'on fait référence à un @xml:id
        else:
            placename.set("ref", f"#{placetext}")
                
    # enfin, on écrit les arbres xml mis à jour dans les bons fichiers
    etree.cleanup_namespaces(tree)
    etree.ElementTree(tree.getroot()).write(
        fpath
        , pretty_print=True
    )

---

### Les entités nommées

Avant de passer aux visualisations, il est temps de travailler sur certaines autres entités nommées de notre corpus: **les destinataires et expéditeur.ice.s des lettres**. Il va s'agir de 
- les identifier
- les dédoublonner 
- les classifier
- pour finir, créer un `particDesc` qui contienne toutes ces infos. 

Si l'identification, le dédoublonnage et la création du `particDesc` sont assez semblables à l'étape précédente, la classification automatique du type d'expéditeur.ice / destinataire va nous introduire à **l'apprentissage machine** et au **traitement automatisé des langues**.

Mais avant tout: **une entité nommée, c'est quoi?** C'est une [expression linguistique référentielle](https://fr.wikipedia.org/wiki/Entit%C3%A9_nomm%C3%A9e), soit quelque chose, qui dans le langage, fait référence à quelque chose. Notion très vague: personne, organisation, œuvre d'art ou tout autre objet du réel sont des entitées nommées.

#### Chaîne de traitement

- on créé `entities`, un dictionnaire associant à l'identifiant 
  unique de chaque expéditeur/destinataire les différentes orthographes 
  de son nom + la langue dans laquelle la lettre comportant son nom
  est écrite
- avec Spacy, on identifie le type de chaque entité nommée: est-ce que
  c'est le nom d'une personne, d'une organisation? si oui, quel type 
  d'organisation?
- on crée un `particDesc` qui contient un `listPerson` avec la liste
  de `person` et un ;`listOrg` avec la liste d'`org` expéditrices/destinataires 
  de lettres dans le corpus, en fonction des données produites par spacy
- on met à jour le `correspDesc` en fonction des informations dans
  le `particDesc`
- on met à jour les fichiers


#### 1. Définition des éléments de base

In [ ]:
# entities associe à un xml:id 3 clés:
# - `name`: la liste des différentes orthographes de ce nom
# - `type`: le type d'entité (personne, organisation...)
# - `lang`: la langue du nom (pour définir le modèle spacy à utiliser)
entities = {}
lang_dict = {}                          # dictionnaire associant à l'xml:id d'une lettre la langue parlée                       
nlp_fr = spacy.load("en_core_web_trf")  # modèle spacy pour l'anglais
nlp_en = spacy.load("fr_dep_news_trf")  # modèle spacy pour le français
particDesc = etree.Element(             # élément xml qui accueillera la liste de personnes 
    "particDesc"
    , nsmap=NS_TEI
)
listPerson = etree.SubElement(          # la liste de personnes
    particDesc
    , "listPerson"
    , nsmap=NS_TEI
)
listOrg = etree.SubElement(             # la liste des organisations
    particDesc
    , "listOrg"
    , nsmap=NS_TEI
)

#### 2. Le premier traitement du corpus

On parse le premier corpus une première fois pour:
- créer un **`listPerson`**, qui fonctionne de la même manière que le `listPlace` créé au dessus
- on **détermine la langue** de chaque lettre, pour choisir le modèle spacy à utiliser

In [ ]:
for fpath in CORPUS:
    tree = etree.parse(fpath, parser=PARSER)
    letter_idx = tree.xpath("./@xml:id", namespaces=NS_TEI)[0]  # letter's @xml:id
        
    # on détecte la langue de chaque lettre. pour ce faire, on extrait
    # le texte de la lettre avec une xpath, on en fait une seule chaîne
    # de caractère et on utilise `detect()` de la librairie `langdetect`
    lang = detect( " ".join(s for s in tree.xpath(".//tei:body//text()", namespaces=NS_TEI)) )
    lang_dict[letter_idx] = lang  # on ajoute la langue à `lang_dict`
    
    # ensuite, on construit le `listPerson`
    for pers in tree.xpath(".//tei:correspAction//tei:persName", namespaces=NS_TEI):
            
        # pour éviter les doublons inutiles, on ajoute pas tous les noms à `place_dict`
        # - on enlève les initiales
        # - on simplifie les chaînes de caractères avec la fonction `xmlid()`
        # - on ajoute pas de doublons
        # - à chaque fois, on ajoute le code de la langue pour déterminer le modèle
        #   à utiliser sur les noms extraits
        
        idx = xmlid(re.sub("((?<=\s)|(?<=^))[A-Z][a-zàâäéèûüùîïì]*\.", "", pers.text))  # enlever initiales
            
        # traitement spécifique pour `na` et équivalents: l'@xml:id défini sera `napartic`
        if re.search("^(inconnu|aucun|na)$", idx):
            if "napartic" not in entities.keys():
                entities["napartic"] = { "name": [pers.text], "type": "", "lang": [lang] }
            elif pers.text not in entities["napartic"]["name"]:
                entities["napartic"]["name"].append(pers.text)
                entities["napartic"]["lang"].append(lang)
        # pour les autres noms:
        else:
            if idx not in entities.keys():
                entities[idx] = { "name": [pers.text], "type": "", "lang": [lang] }
            elif  pers.text not in entities[idx]["name"]:
                entities[idx]["name"].append(pers.text)
                entities[idx]["lang"].append(lang)

print(lang_dict)
print(entities)

#### 3. Détecter le type d'entité dans `entities`

Le dictionnaire `entities` ne contient que des entités nommées. Mais elles peuvent être de types assez différents. Pour limiter le bruit, et mieux comprendre qui écrit à qui, nous allons donc utiliser la reconnaissance d'entités nommées de Spacy, non pas identifier des entités, mais pour classifier celles-ci.

In [ ]:
for k in entities.keys():
    
    # on extrait le mode, soit la langue la plus r
    # épandue pour la liste de lettres avec ce nom
    if len(entities[k]["lang"]) > 0:
        entities[k]["lang"] = mode(entities[k]["lang"])
    else:
        entities[k]["lang"] = ""
    
    # on traite tous les noms associés à un @xml:id
    for name in entities[k]["name"]:
        
        # on sélectionne le bon modèle en fonction du langage. 
        # si le langage détecté n'est ni le français, ni l'anglais,
        # on affiche une erreur mais on utilise le modèle français: la
        # lettre est surement en français 
        if entities[k]["lang"] == "fr":
            doc = nlp_fr(name)
        elif entities[k]["lang"] == "en":
            doc = nlp_en(name)
        else:
            print(f"pas de modèle disponible pour la langue '{entities[k]['lang']}'"
                  + f"détectée dans la lettre: '{letter_idx}'. utilisation du modèle"
                  + " français par défaut 'fr_dep_news_trf'")
            doc = nlp_fr(name)
        
        # pour avoir une idée de la classification produite par spacy
        for ent in doc.ents: 
            print(
                f"source ~ {name}\n"
                , f"détecté ~ {ent.text}\n"
                , f"type ~ {ent.label_} {spacy.explain(ent.label_)}"
                , "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n"
            )

        # enfin, avec spacy, on détecte le type d'entité nommée encodée dans
        # les `correspAction` et on ajoute le label donné à l'entité nommée par spacy 
        # dans `name` à `entities`. spacy produit plusieurs entités nommées => on extrait
        # le mode, soit le type d'entité le + souvent détecté dans notre `name`. vu la 
        # taille de notre corpus c'est un peu inutile, mais bon
        if len(list(doc.ents)) > 0:
            entities[k]["type"] = mode([ ent.label_ for ent in doc.ents ])  # on calcule le mode parmi tous les types d'entités nommées relevées
        else:
            entities[k]["type"] = ""

#### 4. Créer le `particDesc`

On en a fini avec Spacy! La librairie est beaucoup, beaucoup plus puissante que ce pourquoi on l'a utilisée, mais elle a quand même été utile ici: sur un corpus de plus de 100 lettres avec un grand nombre d'auteur.ice.s, il devient difficile d'utiliser seulement de la détection de motifs (chercher "à la main", ou avec des expressions régulières, des termes qui permettent de déterminer si un nom est celui d'une personne ou d'une organisation). On bénéficie avec Spacy d'une énorme base de données de vocabulaire (pleine de biais, bien sûr) qui permet de faire un tri plus fin.

Il s'agit maintenant **d'encoder cette classification en XML-TEI**. Sur le principe, c'est comme la création du `settingDesc`, mais on va faire un peu plus complexe: 
- le `particDesc` aura 2 enfants:
    - un `listPerson` qui contienne toutes les entités détectées comme des personnes
    - un `listOrg`, qui contient toutes les entités détectées comme autre chose que des personnes
- chaque `person` ou `org` aura un attribut `@type` qui contienne le type d'entité détecté par Spacy.
- comme pour les lieux, on va ensuite mettre à jour le `correspAction` pour qu'il contienne un `@ref` aux `@xml:id` des entités dans le `particDesc`.

In [ ]:
for k in entities.keys():
    # déterminer le type d'élément à créer en fonction du type d'entité
    if entities[k]["type"] == "PERSON" or entities[k]["type"] == "":
        parent = listPerson  # l'élément parent: listPerson|listOrg
        el = "person"        # le nom de l'élément lui-même
        elname = "persName"  # si il doit contenir un `persName` ou un `orgName`
    else:
        parent = listOrg
        el = "org"
        elname = "orgName"
    # définir `entity`
    entity = etree.SubElement(
        parent
        , el
        , nsmap=NS_TEI
    )
    
    # définir les attributs
    if entities[k]["type"] != "":
        entity.set("type", entities[k]["type"])  # définir le `@type`: type d'entité
    entity.set("{http://www.w3.org/XML/1998/namespace}id", k)  # définir l'identifiant de `entity`
    
    # définir ses sous-éléments `persName` ou `orgName`, portant les 
    # variantes d'appellations associées à cette entité dans les 
    # différentes lettres du corpus
    for name in entities[k]["name"]:
        etree.SubElement(
            entity
            , elname
            , nsmap=NS_TEI
        ).text = name

#### 5. Mettre à jour les fichiers XML

On reparse (encore et toujours) les fichiers XML pour:
- modifier les `correspAction`: si le `persName` renvoie en fait à une organisation, alors remplacer ce `persName` par un `orgName` (on fait ça à l'aide des différents `persName` / `orgName` dans le `particDesc`)
- ajouter les `@ref` dans le `correspAction` qui font référence aux @xml:id des entités identifiées dans le `particDesc`
- ajouter le `particDesc` au `encodingDesc` du fichier xml
- sauvegarder les arbres mis à jour

In [ ]:
for fpath in CORPUS:
    tree = etree.parse(fpath, parser=PARSER)
        
    # ajout du `particDesc` à l'arbre
    tree.xpath(".//tei:profileDesc", namespaces=NS_TEI)[0].append(particDesc)
        
    # on modifie les correspAction:
    # `corresp` = tous les noms expéditeur.ice.s/destinataires dans le `correspAction`
    for corresp in tree.xpath(".//tei:correspAction/tei:persName", namespaces=NS_TEI):
            
        # on cible l'élément du `particDesc` qui correspond à `corresp`
        for matched_entity in particDesc.xpath(
            f".//*[./text()='{corresp.text}']"
            , namespaces=NS_TEI
        ):
            # on met à jour le tag de `corresp`: si avec spacy, on a détecté que le nom
            # dans `corresp` n'est pas celui d'une personne, alors on change `persName` en `orgName`
            corresp.tag = matched_entity.tag
            # enfin, on ajoute un `@ref` à `corresp` qui pointe vers le bon
            # `persName / org` dans le `particDesc` 
            idx = matched_entity.xpath("./parent::*/@xml:id")[0]
            corresp.set("ref", f"#{idx}")
        
    # on sauvegarde le fichier
    etree.cleanup_namespaces(tree)
    etree.ElementTree(tree.getroot()).write(
        fpath
        , pretty_print=True
    )

## Visualisation

Jusqu'ici, on a déjà beaucoup enrichi notre corpus:
- index des lieux
- géocodage des lieux
- index des expéditeur.ice.s et destinataires, avec classification automatique
- enrichissement des `correspAction` pour créer des liens avec les indexes.

C'est ici qu'on commence à voir l'utilité de la TEI (enfin, j'espère): tous ces enrichissements peuvent être ajoutés après l'encodage du texte et sont encodés à l'intérieur même du document qui contient le texte. On peut donc développer des **chaînes éditoriales** assez complexes.

Pour finir, on va chercher à **produire des visualisations** à partir de notre corpus:
- une visualisation en réseau du corpus: qui écrit à qui? en quel volume?
- une visualisation cartographique: de quelles villes les lettres sont-elles expédiées? Vers quelles villes?

Dans les deux cas, **l'approche va être semblable**:
- parser les fichiers XML pour extraire les informations utiles
- produire des graphes des lieux ou personnes et des relations entre elles.eux
- utiliser ces graphes pour construire des cartes/réseaux.

### Analyse de réseau

Pour faire l'analyse de réseaux, on utilise la librairie Python Pyvis qui produit des graphes interactifs. Le réseau représente les relations entre expéditeur.ice.s et destinataires au sein du corpus Matsutaka. Ce réseau aura les caractéristiques suivantes:
- c'est un graphe orienté (une relation de A vers B =/= une relation de B vers A)
- les nœuds sont les expéditeur.ice.s et destinataires du corpus
- les arrêtes sont les lettres échangées entre elles et eux
- la taille des nœuds et l'épaisseur des arrêtes est déterminée par le volume de lettres

#### 1. Extraction de données

On représente nos nœuds de la façon suivante:

```python
{
    # 1ere entrée
    "@xml:id de la personne": [
        "nom complet qui sera affiché"
        , <nombre de mentions comme expéditeur.ice ou destinataire>
    ]
    # 2e entrée
    , "@xml:id": [
        "nom complet"
        , <décompte>
    ]
}
```

Et nos arrêtes comme ça:

```python
[
    # 1ere relation
    { 
        "from": "@xml:id de l'expéditeur.ice",
        "to": "@xml:id du destinataire"
        "count": <nombre de relations dans ce sens entre expéditeur.ice et destinataire>
    }
    # 2e relation
    , {
        "from": "@xml:id",
        "to":"@xml:id",
        "count": <décompte>
    }
]
```

In [ ]:
# on lit toutes les lettres du corpus, extrait les données pour 
# construire un graphe (noms des expéditeur.ice.s/destinataire et 
# nombre de mention de chacun.e, relation orientées entre expéditeur.ice
# et destinataire et nombre de relations orientées)
nodes = {}  # les nœuds.
edges = []  # les arrêtes
for fpath in CORPUS:
    tree = etree.parse(fpath, parser=PARSER)
        
    # @xml:id de l'expéditeurice et du/de la destinataire
    sender = tree.xpath(
        "//tei:correspAction[@type='sent']/*[not(tei:placeName)][not(tei:date)]/@ref"
        , namespaces=NS_TEI
    )[0].replace("#", "")
    receiver = tree.xpath(
        "//tei:correspAction[@type='received']/*[not(tei:placeName)][not(tei:date)]/@ref"
        , namespaces=NS_TEI
    )[0].replace("#", "")
                
    # en utilisant l'@xml:id, on prend le nom canonique du `particDesc`
    sender_name = tree.xpath(
        f".//tei:particDesc//*[@xml:id='{sender}']/*"
        , namespaces=NS_TEI
    )[0].text
    receiver_name = tree.xpath(
        f".//tei:particDesc//*[@xml:id='{receiver}']/*"
        , namespaces=NS_TEI
    )[0].text

    # on ajoute `sender`/`receiver` à `nodes`. on ne distingue pas le rôle d'expéditeur/destinataire
    if sender not in nodes.keys():
        nodes[sender] = [ sender_name, 1 ]      # 1ere fois que `sender` est identifié comme nœud => créer une nv entrée
    else:
        nodes[sender][1] += 1                   # sinon, on incrémente le compteur d'occurrences pour cette entité
    if receiver not in nodes.keys():
        nodes[receiver] = [ receiver_name, 1 ]  # 1ere fois que `receiver` est identifié comme nœud => nv entrée
    else:
        nodes[receiver][1] +=1                  # sinon, on incrémente le compteur d'occurrences pour cette entité
    
    # on ajoute la relation entre `sender` & `receiver` à `edges`
    # si la relation orientée expéditeurice->destinataire n'existe pas on l'ajoute
    if not any( [sender, receiver] == [edge["from"], edge["to"]] for edge in edges ):
        edges.append({ "from": sender, "to": receiver, "count": 1 })
    # sinon, on récupère le dictionnaire dans `edges` qui décrit la bonne relation et on incrémente son compteur 
    else:
        # on sélectionne l'index de la bonne relation
        for edge in edges:
            if [edge["from"], edge["to"]] == [sender, receiver]:
                idx = edges.index(edge)
        edges[idx]["count"] += 1  # on incrémente son compteur

print(nodes)
print(edges)

#### 2. Créer le réseau

On crée un objet `network` de `pyvis` et in y ajoute les nœuds avec `add_node()` et les arrêtes avec `add_edges()`.

In [ ]:
# créer le réseau
ntw = Network( 
    directed=True                # on travaille avec un graphe orienté
    , bgcolor=COLORS["gold"]    # couleur d'arrière-plan
    , font_color=COLORS["darkgreen"]  # couleur de police
    , filter_menu=True                # un menu pour filtrer par les propriétés des nœuds et arrêtes
    , notebook=True
    , cdn_resources="remote"
)

# ajout des nœuds
for k, v in nodes.items():
    ntw.add_node(
        k                       # l'identifiant du nœud: un @xml:id
        , label=v[0]            # le nom affiché
        , size=v[1]             # la taille du nœud, déterminée par le nb d'occurrences dans le corpus
        , shape="dot"           # la forme
        , color=COLORS["plum"]  # la couleur du nœud
        , title=f"{v[0]} participe à {v[1]} échanges dans le corpus."  # texte à afficher quand on clique s/ le nœud
    )

# ajout des arrêtes
for edge in edges:
    ntw.add_edge(
        edge["from"]           # identifiant du nœud représentant l'expéditeur.ice (défini dans `.add_nodes()`)
        , edge["to"]           # identifiant du nœud représentant le/la destinatairice
        , width=edge["count"]  # l'épaisseur de l'arrête dépend du nombre d'envois
        , title=f"{edge['count']} lettres de { nodes[edge['from']][0] } pour { nodes[edge['to']][0] }"
    )

# bidouiller la physique    
ntw.barnes_hut(overlap=1, gravity=-40000)  # la position des points du réseau)
ntw.toggle_physics(True)  # conseillé par la doc

# afficher le fichier. il y a un bug avec google colab => enregistrer network.html et afficher la page dans un navigateur
ntw.show("network.html")

### Cartographie

Notre cartographie du corpus fonctionne sur d'une manière similaire à la visualisation en graphe: il s'agit de placer des points sur une carte (les nœuds) et ensuite de créer des relations entre eux (les arrêtes). La carte représente les villes d'expédition/destination des lettres du corpus, et les relations entre ces villes. Notre carte aura les caractéristiques suivantes:
- les relations entre villes sont non-orientées (une relation de A à B == une relation de B à A)
- la taille des marqueurs positionnés sur les villes est déterminée par le nombre de lettres qui y sont liées
- l'opacité des arrêtes entre les villes est déterminée par le nombre de lettres qui transitent entre deux villes.

La cartographie est réalisée avec Folium, un port Python de la librairie Leaflet sous Javascript.


#### 1. Extraction de données

Le modèle de données pour nos nœuds est le suivant:
```python
{ 
    "@xml:id 1": <compteur d occurences> 
    , "@xml:id 2": <compteur d occurences>
    , #...
}
```

Le modèle pour le graphe est le suivant:
```python
[ 
    {
        "a": "ville1", 
        "b": "ville2", 
        "count": <compteur d occurrences> 
    }
    , {
        "a": "ville2", 
        "b": "ville3", 
        "count": <compteur d occurrences> 
    }
    , #...  
]
```

In [ ]:
# on parse tous les fichiers XML et on extrait tous les 
# @xml:id des lieux d'expédition/destination du `correspAction`
# afin de construire nodes et edges. dans les deux cas, on ne 
# traite pas les index ayant pour valeur `na`, puisqu'ils ne sont
# pas géoréférencés
nodes = {}
edges = []
geojson_files = [ os.path.splitext(os.path.basename(fp))[0] 
                  for fp in os.listdir(os.path.join(WEB, "json")) ]  # liste de noms de geojson sans extension

for fpath in CORPUS:
    tree = etree.parse(fpath, parser=PARSER)
    indexes = tree.xpath(".//tei:correspAction/tei:placeName/@ref", namespaces=NS_TEI)
    indexes = [ idx.replace("#", "") for idx in indexes ]  # on supprime le `#` au début pour retrouver l'@xml:id
        
    # d'abord, on remplit `nodes`: 
    for idx in indexes:
        if idx not in nodes.keys() and idx != "na":
            nodes[idx] = 1
        elif idx != "na":
            nodes[idx] += 1
                
    # ensuite, on remplit `edges`.
    # on utilise `range` qui à chaque itération émet un index de `edges` =>
    # permet d'itérer à travers tous les items de `edges`.
    # on vérifie dans les 2 sens si `indexes` a déjà une entrée dans `edges`
    # si oui, on incrémente le compteur. sinon, on ajoute une nouvelle entrée
    # à `edges` pour représenter la nouvelle relation entre deux villes.
    # on ne traite une correspondance que si la ville A et la ville B sont géoréférencées
    if all(i in geojson_files for i in indexes):
        sender, receiver = indexes
            
        # si la relation a<->b n'existe pas, on l'ajoute
        if not any( [sender, receiver] == [edge["a"], edge["b"]] for edge in edges ):
            edges.append({ 'a': sender, "b": receiver, "count": 1 })
        # sinon, on incrémente le compteur
        else:
            for i in range(len(edges)):
                if [sender, receiver] == [edges[i]["a"], edges[i]["b"]]:
                    edges[i]["count"] += 1

print(nodes)
print(edges)

Petite complexité supplémentaire: `edges` est pour le moment une relation orientée: il peut y avoir une entrée de la liste où `['a': 'Paris', 'b': 'Kobe']` et une autre où `['a': 'Kobe', 'b': 'Paris']`, chacune avec son compteur. On croise donc ces deux entrées et additionne les compteurs pour que `edges` représente des relations non-dirigées.

D'un point de vue technique, cela revient à intérer deux fois sur `edges` pour faire la comparaison entre les objets émis par les deux itérations.

In [ ]:
edges_undirected = []  # variable pour stocker le graphe non dirigé

for edge in edges:
    for i in range(len(edges)):
        if [ edge["a"], edge["b"] ] == [ edges[i]["b"], edges[i]["a"] ]:
            edge["count"] += edges[i]["count"]
    edges_undirected.append(edge)
edges = edges_undirected

print(edges)

#### 2. Construction de la carte

On crée un objet `folium.Map` et on y ajoute d'abord nos nœuds, puis nos arrêtes.

In [ ]:
map = folium.Map(location=[48.8534951, 2.3483915], tiles="Stamen Toner")
markers = {}  # dictionnaire mappant un @xml:id à un objet `folium.CircleMarker`. sera utilisé pour construire les relations entre les villes
node_titles = {}  # dictionnaire mappant l'@xml:id d'un lieu à son nom lisible
    
# on ajoute d'abord nos nœuds sur la carte
for k, v in nodes.items():
    # on ne traite que les clés qui ont un geojson, soit des infos géographiques
    # attachées.
    if k not in geojson_files:
        print(f"pas de geojson pour l'@xml:id: {k}. ce lieu n'est pas traité")
        continue

    # on charge tous nos fichiers geojson. ils contiennent toutes
    # les infos dont on a besoin (et bien plus)! on s'en sert pour extraire
    # des géocoordonnées et le nom de l'endroit. on pourrait directement afficher
    # un point sur la carte en ajoutant le geojson à notre carte leaflet, mais
    # celui-ci aurait un diamètre fixe (alors qu'on veut un diamètre adapté
    # au nombre de lettres associées au lieu) => on extrait des infos pour
    # créer un `CircleMarker` folium
    with open(os.path.join(WEB, "json", f"{k}.geojson"), mode="r") as fh:
        geojson = json.load(fh)                                   # on ouvre le fichier geojson 
    coordinates = geojson["features"][0]["geometry"]["coordinates"]  # géocoordonnées
    title = geojson["features"][0]["properties"]["display_name"]     # nom complet
        
    node_titles[k] = title
        
    # pour garantir la lisibilité, on représente v (nombre de lettres liées à
    # un endroit) sur une échelle logarithmique et on multiplie cette échelle
    # logarithmique par 5. cela permet d'éviter que les gros marqueurs bloquent
    # toute la carte et que les petits soient invisibles, tout en conservant
    # un ordre de grandeur
    if v > 1:
        vlog = math.log(v) * 5
    else:
        vlog = v * 5
        
    markers[k] = folium.CircleMarker(
        location=[ coordinates[1], coordinates[0] ]          # positionnement
        , radius=vlog                                        # taille du marqueur
        , color=COLORS["plum"]                               # couleur de bordure
        , fill_color=COLORS["gold"]                          # couleur de remplissage
        , fill_opacity=1                                     # opacité
        , popup=f"<b>{title}</b>: <br/><br/> <b>{v}</b> lettres reçues ou expédiées"  # popup s'affichant quand on clicke sur le marker
    )
    markers[k].add_to(map)
    
# ensuite, on ajoute nos arrêtes: les relations entre 2 villes
maxcount = max([ e["count"] for e in edges ])  # +gd nombre de relations
for edge in edges:
    # on définit l'opacité en fonction du nombre de lettres envoyées: 
    # 0.3 + 0.7 x <proportion du nombre de lettres entre les 2 villes actuelles 
    #              par rapport au nombre maximal de lettres envoyées entre 2 villes>
    opacity = (edge["count"] / maxcount) * 0.7 + 0.3
    folium.PolyLine(
        locations=[ markers[edge["a"]].location, markers[edge["b"]].location ]  # positions des 2 villes
        , color=COLORS["plum"]
        , stroke=5
        , opacity=opacity
        , fillColor=COLORS["plum"]
        , fillOpacity=opacity
        , popup=f"<b>{edge['count']}</b> lettres échangées entre <b>{node_titles[edge['a']]}</b> et <b>{node_titles[edge['b']]}</b>"
        , tooltip=f"<b>{node_titles[edge['a']]}</b> <br/><br/> <b>{node_titles[edge['b']]}</b>"
    ).add_to(map)

map  # tadaaaaaa
# map.save(os.path.join(WEB, "map.html"))  # pour la voir en grand écran, décommenter puis aller ouvrir ce fichier

## Pour conclure

En partant d'un corpus en XML tout simple, on voit comment on peut arriver très vite à mettre au point une **pipeline d'édition scientifique**, avec des améliorations progressives et automatiques de l'encodage. Pour celles-ci, il n'est pas directement nécessaire de déployer l'artillerie lourde, faire un encodage plus qualitatif de nos métadonnées est possible de façon relativement simple. Reste ensuite à traiter le corps des lettres, que l'on a sagement laissé de côté.

Quand on croise l'édition TEI avec des visualisations, on voit que des résultats assez arrivent assez facilement, et permettent de voir de façon synthétique l'évolution du corpus. Par contre, **la temporalité du corpus** n'a pas été prise en compte. Il serait intéressant de voir comment la spatialité et les interaction évoluent au fil du temps, mais mettre des filtres temporels aurait demandé de rajouter du Javascript etc, ce qui aurait été un peu compliqué.

Si vous avez encore le courage: dans [`src/txt2xml.py`](https://github.com/paulhectork/cours_ens2023_xmltei/blob/main/src/txt2xml.py) est présentée une pipeline pour la production automatique du corpus en TEI à partir d'une version texte brut. C'est avec cette pipeline que le corpus sur lequel s'est basé cet atelier a été construite : )